# Send Mail Using Databricks

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CreateTableAndSaveAsCSV") \
    .getOrCreate()

In [0]:
data = [
    (1, "John Doe", 30, "New York"),
    (2, "Jane Doe", 25, "Los Angeles"),
    (3, "Sam Brown", 28, "Chicago")
]

columns = ["ID", "Name", "Age", "City"]

df = spark.createDataFrame(data, columns)
df.show()

+---+---------+---+-----------+
| ID|     Name|Age|       City|
+---+---------+---+-----------+
|  1| John Doe| 30|   New York|
|  2| Jane Doe| 25|Los Angeles|
|  3|Sam Brown| 28|    Chicago|
+---+---------+---+-----------+



In [0]:
output_path = "dbfs:/FileStore/tables/csvfile.csv"
df.write.csv(output_path, header=True)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3953264074371876>:2
      1 output_path = "dbfs:/FileStore/tables/csvfile.csv"
----> 2 df.write.csv(output_path, header=True)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1798, in DataFrameWriter.csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
   1779 self.mode

In [0]:
df_csv = spark.read.csv(output_path, header=True, inferSchema=True)
df_csv.show()

In [0]:
#Email Paramaeters
sender_email = 'arvndh001@outlook.com'
receiver_email = 'arvndh001@gmail.com'
password = 'dqtkrouikyihjqpz'

In [0]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

- smtplib: This library handles sending the email via SMTP (Simple Mail Transfer Protocol).
- MIMEMultipart: Used to create an email message that can contain multiple parts (e.g., text and attachments).
- MIMEBase: A base class for handling the attachment in the email.
- MIMEText: Used to create the text part of the email.
- encoders: Used to encode the attachment in base64 format so it can be sent via email.

In [0]:
# List all directories at the root of DBFS
root_directories = dbutils.fs.ls('dbfs:/')

# Define system directories that should not be deleted
protected_directories = ['dbfs:/FileStore/', 'dbfs:/databricks-datasets/', 'dbfs:/databricks-results/', 'dbfs:/mnt/']

# Loop through each directory and delete its contents if not protected
for directory in root_directories:
    path = directory.path
    if path not in protected_directories:
        print(f"Deleting contents of: {path}")
        dbutils.fs.rm(path, recurse=True)
    else:
        print(f"Skipping protected system directory: {path}")


Skipping protected system directory: dbfs:/FileStore/
Skipping protected system directory: dbfs:/databricks-datasets/
Skipping protected system directory: dbfs:/databricks-results/
Skipping protected system directory: dbfs:/mnt/


In [0]:
# List all files and directories inside /FileStore/tables/
files_to_delete = dbutils.fs.ls('/FileStore/tables/')

# Loop through each file and delete it
for file in files_to_delete:
    path = file.path
    print(f"Deleting: {path}")
    dbutils.fs.rm(path, recurse=True)


Deleting: dbfs:/FileStore/tables/csvfile.csv/
Deleting: dbfs:/FileStore/tables/product.csv
Deleting: dbfs:/FileStore/tables/sam.csv


In [0]:
%fs
ls /databricks-datasets/COVID/covid-19-data/us.csv

path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/covid-19-data/us.csv,us.csv,10269,1615898768000


In [0]:
dbutils.fs.cp('/databricks-datasets/COVID/covid-19-data/us.csv', 'file:/tmp/us.csv')


Out[162]: True

In [0]:
import os

# Specify the local directory
local_directory = '/tmp/'

# List files in the directory
files = os.listdir(local_directory)
print(files)


['hsperfdata_root', '.ICE-unix', 'systemd-private-84bda213a5724075bd0aaaf815995056-systemd-logind.service-YujWqj', 'csvfile.csv', 'chauffeur-daemon-params', '.us.csv.crc', 'python_lsp_logs', 'RtmpKOZ18V', 'us.csv', '.part-00000.csv.crc', 'driver-daemon.pid', '.font-unix', 'driver-daemon-params', 'driver-env.sh', 'part-00000.csv', 'chauffeur-daemon.pid', 'custom-spark.conf', 'Rserv', 'systemd-private-84bda213a5724075bd0aaaf815995056-ntp.service-HISl1e', 'systemd-private-84bda213a5724075bd0aaaf815995056-apache2.service-ojHt6e', '.XIM-unix', 'tmp.7PVMXAvUol', 'systemd-private-84bda213a5724075bd0aaaf815995056-systemd-resolved.service-SYwQBf', 'chauffeur-env.sh', '.Test-unix', '.X11-unix']


In [0]:
#Attachment Paramaters
attachment_path = '/tmp/us.csv'

In [0]:
#create a multipart message and set headers
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = 'CSV Attachment Test'
#Custimize the body message
body = 'Hello, Please go through the CSV attachment'

#Convert the body to a MIME compatable string
body = MIMEText(body)
#Attach it to your main message
message.attach(body)

In [0]:
#Open the attachment file in bynary
with open(attachment_path, 'rb') as attachment:
    #Add file as appliction/octet-stream
    part = MIMEBase('application', 'ocet-stream')
    part.set_payload((attachment).read())

#Encode file in ASCII characters to send by email
encoders.encode_base64(part)

In [0]:
#Add header with pdf name
part.add_header(
    'Content_Disposition',
    f'attachment: filename = {attachment_path}'
)

In [0]:
#Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

In [0]:
# Connect to the Outlook SMTP server and send the email
with smtplib.SMTP('smtp-mail.outlook.com',587) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)

In [0]:
# Log in to Gmail SMTP server using secure context and send email
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)